In [1]:
import pandas as pd
from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders
from tqdm import tqdm
from typing import List, Dict
import json
from joblib import delayed, Parallel
import itertools
from itertools import chain
import pickle as pkl

In [61]:
df_results_from_query = pd.DataFrame(pkl.load(open("results_with_names_doi_relationship.pkl", "rb")))
df_results_from_query.iloc[1518] #["title"]

project_id                                                      96/05497-3
orcids                   [https://orcid.org/0000-0001-5203-4366, https:...
title                                        Pesquisa em visão cibernética
summary                  O presente projeto visa suportar as atividades...
keywords                 [Inteligência artificial, Visão computacional,...
related_dois                                                            []
pubs_names               [Optimized approach to multiscale skeleton gen...
name_doi_relationship    {'Optimized approach to multiscale skeleton ge...
Name: 1518, dtype: object

In [3]:
papers_names_with_dois = [eval(str(x)) for x in df_results_from_query["name_doi_relationship"]]

In [4]:
not_recovered_papers = []
recovered_papers = []
for x, (_, row) in zip(papers_names_with_dois, df_results_from_query.iterrows()): 
    for k,v in x.items():
        if isinstance(v, list):
            not_recovered_papers.append((k, row))
        elif isinstance(v, str):
            if v == 'https://www.doi.org/' or v == 'http://www.doi.org/':
                not_recovered_papers.append((k, row))
            else:
                recovered_papers.append((k, v))

In [5]:
len(recovered_papers) / (len(not_recovered_papers) + len(recovered_papers))

0.42898081235983054

In [6]:
recovered_papers

[('State Disaggregation for Dynamic Scaling of Network Functions',
  'https://www.doi.org/10.1109/TNET.2023.3282562'),
 ('Analysis of functional connectivity using machine learning and deep learning in different data modalities from individuals with schizophrenia',
  'https://www.doi.org/10.1088/1741-2552/acf734'),
 ('Neural architecture search with interpretable meta-features and fast predictors',
  'https://www.doi.org/10.1016/j.ins.2023.119642'),
 ('Slicing who slices: Anonymization quality evaluation on deployment, privacy, and utility in mix-zones',
  'https://www.doi.org/10.1016/j.comnet.2023.110007'),
 ('Towards augmented and mixed reality on future mobile networks',
  'https://www.doi.org/10.1007/s11042-023-15301-4'),
 ('On strategies to help reduce contamination on public transit: a multilayer network approach',
  'https://www.doi.org/10.1007/s41109-023-00562-7'),
 ('A Novel Mechanism for Misbehavior Detection in Vehicular Networks',
  'https://www.doi.org/10.1109/ACCESS.2023.

In [7]:
recovered_papers_open_alex = []
for x in tqdm(recovered_papers, total=len(recovered_papers)):
    try:
        recovered_papers_open_alex.append(Works()[x[1]])
    except:
        pass

100%|█████████████████████████████| 3443/3443 [42:29<00:00,  1.35it/s]


In [8]:
len(recovered_papers_open_alex) / (len(not_recovered_papers) + len(recovered_papers))

0.41913780214303514

In [9]:
not_recovered_papers_recovering_by_title = []
for p in tqdm(not_recovered_papers, total=len(not_recovered_papers)):
    try:
        not_recovered_papers_recovering_by_title.append(
            (p[0], Works().search_filter(title=p[0]).get()[0])
        )
    except:
        pass

100%|███████████████████████████| 4583/4583 [1:06:24<00:00,  1.15it/s]


In [7]:
import pickle as pkl
#pkl.dump(recovered_papers_open_alex, open("recovered_papers_open_alex.pkl", "wb"))
#pkl.dump(not_recovered_papers_recovering_by_title, open("not_recovered_papers_recovering_by_title.pkl", "wb"))

In [8]:
recovered_papers_open_alex = pkl.load(open("recovered_papers_open_alex.pkl", "rb"))
not_recovered_papers_recovering_by_title = pkl.load(open("not_recovered_papers_recovering_by_title.pkl", "rb"))

In [9]:
(len(recovered_papers_open_alex)+len(not_recovered_papers_recovering_by_title)) / (len(not_recovered_papers) + len(recovered_papers))

0.9438076252180414

In [15]:
not_recovered_papers_recovering_by_title[0][0]

'Pruning and Early-Exit Co-Optimization for CNN Acceleration on FPGAs'

In [21]:
all_papers = recovered_papers_open_alex + [x[1] for x in not_recovered_papers_recovering_by_title]

In [34]:
all_papers[0]

{'id': 'https://openalex.org/W4380369136',
 'doi': 'https://doi.org/10.1109/tnet.2023.3282562',
 'title': 'State Disaggregation for Dynamic Scaling of Network Functions',
 'display_name': 'State Disaggregation for Dynamic Scaling of Network Functions',
 'publication_year': 2023,
 'publication_date': '2023-01-01',
 'ids': {'openalex': 'https://openalex.org/W4380369136',
  'doi': 'https://doi.org/10.1109/tnet.2023.3282562'},
 'language': 'en',
 'primary_location': {'is_oa': False,
  'landing_page_url': 'https://doi.org/10.1109/tnet.2023.3282562',
  'pdf_url': None,
  'source': {'id': 'https://openalex.org/S62238642',
   'display_name': 'IEEE ACM Transactions on Networking',
   'issn_l': '1063-6692',
   'issn': ['1063-6692', '1558-2566'],
   'is_oa': False,
   'is_in_doaj': False,
   'host_organization': 'https://openalex.org/P4310319808',
   'host_organization_name': 'Institute of Electrical and Electronics Engineers',
   'host_organization_lineage': ['https://openalex.org/P4310319808'],

In [37]:
only_dois_found_on_oa = []
for paper in all_papers:
    doi = paper["doi"]
    title = paper["title"]
    authors = [(a["author"]["orcid"], a["author"]["display_name"]) for a in paper["authorships"] 
     if "author" in a and "orcid" in a["author"] and a["author"]["orcid"] != None]
    authors_orcid = [a[0] for a in authors]
    authors_names = [a[1] for a in authors]
    only_dois_found_on_oa.append(
        {
            "doi": doi,
            "title": title,
            "authors_orcid": authors_orcid,
            "authors_names": authors_names,
        }
    )
only_dois_found_on_oa = pd.DataFrame(only_dois_found_on_oa)
only_dois_found_on_oa

,doi,title,authors_orcid,authors_names
0,https://doi.org/10.1109/tnet.2023.3282562,State Disaggregation for Dynamic Scaling of Ne...,"[https://orcid.org/0000-0002-3481-4251, https:...","[Fabrício Franco Carvalho, Ronaldo A. Ferreira..."
1,https://doi.org/10.1088/1741-2552/acf734,Analysis of functional connectivity using mach...,"[https://orcid.org/0000-0003-4708-1330, https:...","[Caroline L. Alves, Thaise G. L. de O. Toutain..."
2,https://doi.org/10.1016/j.ins.2023.119642,Neural architecture search with interpretable ...,"[https://orcid.org/0000-0001-6586-6401, https:...","[Gean Trindade Pereira, Iury Batista de Andrad..."
3,https://doi.org/10.1016/j.comnet.2023.110007,Slicing who slices: Anonymization quality eval...,"[https://orcid.org/0000-0001-7020-1703, https:...","[Ekler Paulino de Mattos, Augusto C. S. A. Dom..."
4,https://doi.org/10.1007/s11042-023-15301-4,Towards augmented and mixed reality on future ...,"[https://orcid.org/0000-0003-4537-5183, https:...","[Luís Fernando de Souza Cardoso, Bruno Yuji Li..."
...,...,...,...,...
7570,https://doi.org/10.1007/s00453-007-9006-9,Algorithms for Maximum Independent Set in Conv...,[https://orcid.org/0000-0003-0242-5603],[Jacyra Ramos Soares]
7571,https://doi.org/10.1017/s0963548301004989,On the Circuit Cover Problem for Mixed Graphs,"[https://orcid.org/0000-0003-4462-3325, https:...","[Orlando Lee, Yoshiko Wakabayashi]"
7572,https://doi.org/10.1007/pl00009828,Induced Ramsey Numbers,[https://orcid.org/0000-0001-7841-157X],[Yoshiharu Kohayakawa]
7573,https://doi.org/10.1016/s0925-2312(01)00628-2,Realistic neuromorphic models and their applic...,[https://orcid.org/0000-0002-4428-9745],[Regina Célia Coelho]


In [62]:
df = pd.read_csv("fapesp_subset_ciencias_da_computacao.csv", delimiter=";", index_col=False)
df = df[df["Beneficiário"].notna() | df["Pesquisador Responsável"].notna()].reset_index(drop=True)
df

/tmp/ipykernel_99594/3775867619.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv("fapesp_subset_ciencias_da_computacao.csv", delimiter=";", index_col=False)


,N. Processo,Título (Português),Título (Inglês),Beneficiário,Instituição,Cidade Instituição,Instituição Parceira,Empresa,Município,Pesquisador Responsável,...,Assuntos,Data de Início,Data de Término,Acordo(s)/Convênio(s) de Cooperação com a FAPESP,Instituições no Exterior,País (Inst. Ext. / Convênio),Pesquisador responsável no exterior,Resumo (Português),Resumo (Inglês),Processos Vinculados
0,23/04895-4,O número de Ramsey relativo a arestas e grafos...,Size-Ramsey numbers and Ramsey-finiteness of g...,Meysam Miralaei,Universidade de São Paulo (USP). Instituto de ...,São Paulo,NaN,NaN,NaN,Yoshiharu Kohayakawa,...,Combinatória:Teoria de Ramsey:Grafos:Métodos p...,2024-05-01,2026-04-30,NaN,NaN,NaN,NaN,Este projeto tem foco em problemas sobre o núm...,This project focuses on numerical problems inv...,NaN
1,23/17002-8,Verificação de identidade de Usuários em redes...,User Identity Verification in CBDC Networks Us...,João Pedro Alonso Almeida,Universidade de São Paulo (USP). Instituto de ...,São Carlos,NaN,NaN,NaN,Jó Ueyama,...,Blockchain,2024-03-25,2024-07-24,NaN,Queensland University Of Technology,Austrália,Raja Jurdak,Uma Moeda Digital de Banco Central (CBDC) repr...,A Central Bank Digital Currency (CBDC) represe...,NaN
2,23/13688-2,An Autoencoder model for dealing with missing ...,Um modelo Autoencoder para lidar com dados aus...,Arthur Dantas Mangussi,Ministério da Defesa (Brasil). Instituto Tecno...,São José dos Campos,NaN,NaN,NaN,Ana Carolina Lorena,...,Aprendizado computacional:Pré-processamento de...,2024-03-15,2024-09-14,NaN,Universidade de Coimbra (UC),Portugal,Pedro Henriques Abreu,Vários problemas podem deteriorar a qualidade ...,Various issues can deteriorate data quality in...,NaN
3,23/10398-3,HUB FUNDOS: plataforma computacional para a pa...,HUB FUNDOS: computational platform for the sta...,Aloisio Mota Rodrigues Junior,NaN,NaN,NaN,RTM Infraestrutura em Tecnologia da Informação...,São Paulo,Aloisio Mota Rodrigues Junior,...,Processamento de linguagem natural:Transformaç...,2024-03-01,2024-11-30,NaN,NaN,NaN,NaN,"Nos últimos dez anos, a indústria de Fundos no...","Over the last ten years, the Funds industry in...",NaN
4,23/06461-1,O uso de videogames no monitoramento e na manu...,The use of video games in monitoring and maint...,Fabio Ota,NaN,NaN,NaN,Eliza Massako Akiyama Ota,São Paulo,Fabio Ota,...,Inteligência artificial:Jogos de computador:Lo...,2024-03-01,2025-02-28,NaN,NaN,NaN,NaN,O envelhecimento populacional é um fenômeno ob...,Population aging is a phenomenon observed worl...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7492,92/01292-7,Desenvolvimento e implementação de algoritmo p...,NaN,Edson Roberto Minatel,"Ministério da Agricultura, Pecuária e Abasteci...",São Carlos,NaN,NaN,NaN,Paulo Estevão Cruvinel,...,Transformada de Fourier:Algoritmos,1992-07-01,1993-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7493,92/01362-5,NaN,A scheduling stratagy for a distributed hard r...,Maurício Ferreira Magalhães,Universidade Estadual de Campinas (UNICAMP). F...,Campinas,NaN,NaN,NaN,Maurício Ferreira Magalhães,...,NaN,1992-06-23,1992-06-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7494,92/01118-7,Computação paralela em álgebra linear numérica,NaN,Edna Yoshiko Senzako,Universidade Estadual Paulista (UNESP). Campus...,São José do Rio Preto,NaN,NaN,NaN,Odelar Leite Linhares,...,Álgebra linear,1992-06-01,1993-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7495,92/00542-0,Autômatos finitos: teoria e prática,NaN,Imre Simon,Universidade de São Paulo (USP). Instituto de ...,São Paulo,NaN,NaN,NaN,Imre Simon,...,Autômatos finitos:Compressão de dados:Entropia...,1992-05-30,1992-07-13,NaN,Université Paris Diderot - Paris 7,França,Dominique Perrin,Pretendo prosseguir e aprofundar as minhas pes...,NaN,NaN


In [39]:
#df = df.merge(df_results_from_query[["project_id", "orcids", "name_doi_relationship"]].rename(columns={"project_id": "N. Processo"}), on="N. Processo", how="left")

In [63]:
#df_results_from_query.merge(only_dois_found_on_oa, on="title")

,project_id,orcids,title,summary,keywords,related_dois,pubs_names,name_doi_relationship,doi,authors_orcid,authors_names


In [64]:
only_dois_found_on_oa = only_dois_found_on_oa.drop_duplicates("title").reset_index(drop=True)

In [65]:
all_names = []
all_orcids = []
for _, row_fapesp in df_results_from_query.iterrows():
    names = eval(str(row_fapesp["pubs_names"]))
    df_in_common = only_dois_found_on_oa[only_dois_found_on_oa["title"].isin(names)]
    row_names = []
    row_orcids = []
    for _, row_openalex in df_in_common.iterrows():
        row_orcids.extend(eval(str(row_openalex["authors_orcid"])))
        row_names.extend(eval(str(row_openalex["authors_names"])))
    all_names.append(row_names)
    all_orcids.append(row_orcids)
df_results_from_query["all_names_oa"] = all_names
df_results_from_query["all_orcids_oa"] = all_orcids

In [66]:
df_results_from_query

,project_id,orcids,title,summary,keywords,related_dois,pubs_names,name_doi_relationship,all_names_oa,all_orcids_oa
0,23/00812-7,"[https://orcid.org/0000-0002-9144-7187, https:...",Aceleração de aplicações sensíveis a atrasos e...,Aplicações em nuvem são complexas e normalment...,"[Redes de computadores, Computação em nuvem, F...",[https://www.doi.org/10.1109/TNET.2023.3282562],[State Disaggregation for Dynamic Scaling of N...,{'State Disaggregation for Dynamic Scaling of ...,"[Fabrício Franco Carvalho, Ronaldo A. Ferreira...","[https://orcid.org/0000-0002-3481-4251, https:..."
1,20/09835-1,"[http://orcid.org/0000-0002-4765-6459, http://...",IARA - Inteligência Artificial Recriando Ambie...,O Centro de Pesquisa Aplicada em Inteligência ...,"[Aprendizado computacional, Cidades inteligent...","[https://www.doi.org/10.1088/1741-2552/acf734,...",[Analysis of functional connectivity using mac...,{'Analysis of functional connectivity using ma...,"[Caroline L. Alves, Thaise G. L. de O. Toutain...","[https://orcid.org/0000-0003-4708-1330, https:..."
2,23/00721-1,"[https://orcid.org/0000-0003-4523-6466, https:...",Quantificação de incerteza em aprendizado fede...,O projeto de pesquisa denominado Quantifying U...,"[Aprendizado federado, Redes bayesianas, Apren...",[https://www.doi.org/10.1016/j.comnet.2023.110...,[Slicing who slices: Anonymization quality eva...,{'Slicing who slices: Anonymization quality ev...,"[Ekler Paulino de Mattos, Augusto C. S. A. Dom...","[https://orcid.org/0000-0001-7020-1703, https:..."
3,22/14503-3,"[https://orcid.org/0000-0002-2174-2762, https:...",Serviços de transporte inteligentes para as re...,"As futuras redes 6G são esperadas para 2030, q...","[Redes de computadores, Transmissão de dados, ...",[https://www.doi.org/10.1007/s11042-023-15301-4],[Towards augmented and mixed reality on future...,{'Towards augmented and mixed reality on futur...,"[Luís Fernando de Souza Cardoso, Bruno Yuji Li...","[https://orcid.org/0000-0003-4537-5183, https:..."
4,23/00148-0,"[https://orcid.org/0000-0001-6994-8076, https:...",SocialNet: sensoriamento social para alavancar...,"Novas fontes massivas de dados, como o Waze e ...","[Computação urbana, Caracterização, Mineração ...",[https://www.doi.org/10.1007/s41109-023-00562-7],[On strategies to help reduce contamination on...,{'On strategies to help reduce contamination o...,"[Fernanda R. Gubert, Mauro Fonseca, Anelise Mu...","[https://orcid.org/0000-0003-3315-8026, https:..."
...,...,...,...,...,...,...,...,...,...,...
1517,96/07200-8,"[https://orcid.org/0000-0002-6022-0270, https:...","Abordagens iterativas em problemas inversos, c...",Serão abordadas três linhas de pesquisa conect...,"[Algoritmos, Algoritmos genéticos, Computação ...",[],[Inversão numérica de distribuições bidimensio...,{'Inversão numérica de distribuições bidimensi...,"[Fernando M. Ramos, Jonas C. Carvalho]","[https://orcid.org/0000-0001-5321-9805, https:..."
1518,96/05497-3,"[https://orcid.org/0000-0001-5203-4366, https:...",Pesquisa em visão cibernética,O presente projeto visa suportar as atividades...,"[Inteligência artificial, Visão computacional,...",[],[Optimized approach to multiscale skeleton gen...,{'Optimized approach to multiscale skeleton ge...,"[Luciano da F. Costa, Maysa Santos Barbosa, Ri...","[https://orcid.org/0000-0001-5203-4366, https:..."
1519,96/04505-2,"[http://orcid.org/0000-0001-7841-157X, http://...",Aspectos estruturais e algorítmicos de objetos...,O objeto central da pesquisa proposta neste pr...,"[Algoritmos, Combinatória, Poliedros]",[https://www.doi.org/],"[Hereditary properties of triple systems, A de...",{'Hereditary properties of triple systems': []...,"[Yoshiharu Kohayakawa, Hein van der Holst, Jos...","[https://orcid.org/0000-0001-7841-157X, https:..."
1520,94/04691-5,"[https://orcid.org/0000-0001-5203-4366, https:...",Caracterização e análise de forma e textura ut...,O objetivo principal do corrente projeto consi...,"[Visão computacional, Análise de imagens, Text...",[]

In [75]:
df_results_from_query

,project_id,orcids,title,summary,keywords,related_dois,pubs_names,name_doi_relationship,all_names_oa,all_orcids_oa
0,23/00812-7,"[https://orcid.org/0000-0002-9144-7187, https:...",Aceleração de aplicações sensíveis a atrasos e...,Aplicações em nuvem são complexas e normalment...,"[Redes de computadores, Computação em nuvem, F...",[https://www.doi.org/10.1109/TNET.2023.3282562],[State Disaggregation for Dynamic Scaling of N...,{'State Disaggregation for Dynamic Scaling of ...,"[Fabrício Franco Carvalho, Ronaldo A. Ferreira...","[https://orcid.org/0000-0002-3481-4251, https:..."
1,20/09835-1,"[http://orcid.org/0000-0002-4765-6459, http://...",IARA - Inteligência Artificial Recriando Ambie...,O Centro de Pesquisa Aplicada em Inteligência ...,"[Aprendizado computacional, Cidades inteligent...","[https://www.doi.org/10.1088/1741-2552/acf734,...",[Analysis of functional connectivity using mac...,{'Analysis of functional connectivity using ma...,"[Caroline L. Alves, Thaise G. L. de O. Toutain...","[https://orcid.org/0000-0003-4708-1330, https:..."
2,23/00721-1,"[https://orcid.org/0000-0003-4523-6466, https:...",Quantificação de incerteza em aprendizado fede...,O projeto de pesquisa denominado Quantifying U...,"[Aprendizado federado, Redes bayesianas, Apren...",[https://www.doi.org/10.1016/j.comnet.2023.110...,[Slicing who slices: Anonymization quality eva...,{'Slicing who slices: Anonymization quality ev...,"[Ekler Paulino de Mattos, Augusto C. S. A. Dom...","[https://orcid.org/0000-0001-7020-1703, https:..."
3,22/14503-3,"[https://orcid.org/0000-0002-2174-2762, https:...",Serviços de transporte inteligentes para as re...,"As futuras redes 6G são esperadas para 2030, q...","[Redes de computadores, Transmissão de dados, ...",[https://www.doi.org/10.1007/s11042-023-15301-4],[Towards augmented and mixed reality on future...,{'Towards augmented and mixed reality on futur...,"[Luís Fernando de Souza Cardoso, Bruno Yuji Li...","[https://orcid.org/0000-0003-4537-5183, https:..."
4,23/00148-0,"[https://orcid.org/0000-0001-6994-8076, https:...",SocialNet: sensoriamento social para alavancar...,"Novas fontes massivas de dados, como o Waze e ...","[Computação urbana, Caracterização, Mineração ...",[https://www.doi.org/10.1007/s41109-023-00562-7],[On strategies to help reduce contamination on...,{'On strategies to help reduce contamination o...,"[Fernanda R. Gubert, Mauro Fonseca, Anelise Mu...","[https://orcid.org/0000-0003-3315-8026, https:..."
...,...,...,...,...,...,...,...,...,...,...
1517,96/07200-8,"[https://orcid.org/0000-0002-6022-0270, https:...","Abordagens iterativas em problemas inversos, c...",Serão abordadas três linhas de pesquisa conect...,"[Algoritmos, Algoritmos genéticos, Computação ...",[],[Inversão numérica de distribuições bidimensio...,{'Inversão numérica de distribuições bidimensi...,"[Fernando M. Ramos, Jonas C. Carvalho]","[https://orcid.org/0000-0001-5321-9805, https:..."
1518,96/05497-3,"[https://orcid.org/0000-0001-5203-4366, https:...",Pesquisa em visão cibernética,O presente projeto visa suportar as atividades...,"[Inteligência artificial, Visão computacional,...",[],[Optimized approach to multiscale skeleton gen...,{'Optimized approach to multiscale skeleton ge...,"[Luciano da F. Costa, Maysa Santos Barbosa, Ri...","[https://orcid.org/0000-0001-5203-4366, https:..."
1519,96/04505-2,"[http://orcid.org/0000-0001-7841-157X, http://...",Aspectos estruturais e algorítmicos de objetos...,O objeto central da pesquisa proposta neste pr...,"[Algoritmos, Combinatória, Poliedros]",[https://www.doi.org/],"[Hereditary properties of triple systems, A de...",{'Hereditary properties of triple systems': []...,"[Yoshiharu Kohayakawa, Hein van der Holst, Jos...","[https://orcid.org/0000-0001-7841-157X, https:..."
1520,94/04691-5,"[https://orcid.org/0000-0001-5203-4366, https:...",Caracterização e análise de forma e textura ut...,O objetivo principal do corrente projeto consi...,"[Visão computacional, Análise de imagens, Text...",[]

In [69]:
df["all_researchers"] = df[["N. Processo", "Beneficiário", "Pesquisador Responsável", 'Pesquisador responsável no exterior', 'Pesquisadores Associados', 'Supervisor', 'Pesquisadores Principais', 'Pesquisador Visitante']] \
.apply(lambda x: set([x["Beneficiário"], x["Pesquisador Responsável"], x["Pesquisador responsável no exterior"], x["Pesquisadores Associados"], x['Supervisor'], x['Pesquisadores Principais'], x['Pesquisador Visitante']]) , axis=1)

In [77]:
df_results_from_query_simple = df_results_from_query.merge(df[["N. Processo", "all_researchers"]].rename(columns={"N. Processo": "project_id"}), on="project_id", how="left")
df_results_from_query_simple

,project_id,orcids,title,summary,keywords,related_dois,pubs_names,name_doi_relationship,all_names_oa,all_orcids_oa,all_researchers
0,23/00812-7,"[https://orcid.org/0000-0002-9144-7187, https:...",Aceleração de aplicações sensíveis a atrasos e...,Aplicações em nuvem são complexas e normalment...,"[Redes de computadores, Computação em nuvem, F...",[https://www.doi.org/10.1109/TNET.2023.3282562],[State Disaggregation for Dynamic Scaling of N...,{'State Disaggregation for Dynamic Scaling of ...,"[Fabrício Franco Carvalho, Ronaldo A. Ferreira...","[https://orcid.org/0000-0002-3481-4251, https:...","{Italo Fernando Scotá Cunha, nan, Ronaldo Alve..."
1,20/09835-1,"[http://orcid.org/0000-0002-4765-6459, http://...",IARA - Inteligência Artificial Recriando Ambie...,O Centro de Pesquisa Aplicada em Inteligência ...,"[Aprendizado computacional, Cidades inteligent...","[https://www.doi.org/10.1088/1741-2552/acf734,...",[Analysis of functional connectivity using mac...,{'Analysis of functional connectivity using ma...,"[Caroline L. Alves, Thaise G. L. de O. Toutain...","[https://orcid.org/0000-0003-4708-1330, https:...",{André Carlos Ponce de Leon Ferreira de Carval...
2,23/00721-1,"[https://orcid.org/0000-0003-4523-6466, https:...",Quantificação de incerteza em aprendizado fede...,O projeto de pesquisa denominado Quantifying U...,"[Aprendizado federado, Redes bayesianas, Apren...",[https://www.doi.org/10.1016/j.comnet.2023.110...,[Slicing who slices: Anonymization quality eva...,{'Slicing who slices: Anonymization quality ev...,"[Ekler Paulino de Mattos, Augusto C. S. A. Dom...","[https://orcid.org/0000-0001-7020-1703, https:...","{Heitor Soares Ramos Filho, nan, nan, Alejandr..."
3,22/14503-3,"[https://orcid.org/0000-0002-2174-2762, https:...",Serviços de transporte inteligentes para as re...,"As futuras redes 6G são esperadas para 2030, q...","[Redes de computadores, Transmissão de dados, ...",[https://www.doi.org/10.1007/s11042-023-15301-4],[Towards augmented and mixed reality on future...,{'Towards augmented and mixed reality on futur...,"[Luís Fernando de Souza Cardoso, Bruno Yuji Li...","[https://orcid.org/0000-0003-4537-5183, https:...","{nan, nan, Toktam Mahmoodi, Bruno Yuji Lino Ki..."
4,23/00148-0,"[https://orcid.org/0000-0001-6994-8076, https:...",SocialNet: sensoriamento social para alavancar...,"Novas fontes massivas de dados, como o Waze e ...","[Computação urbana, Caracterização, Mineração ...",[https://www.doi.org/10.1007/s41109-023-00562-7],[On strategies to help reduce contamination on...,{'On strategies to help reduce contamination o...,"[Fernanda R. Gubert, Mauro Fonseca, Anelise Mu...","[https://orcid.org/0000-0003-3315-8026, https:...","{nan, Daniel Silver - Leandro Aparecido Villas..."
...,...,...,...,...,...,...,...,...,...,...,...
1517,96/07200-8,"[https://orcid.org/0000-0002-6022-0270, https:...","Abordagens iterativas em problemas inversos, c...",Serão abordadas três linhas de pesquisa conect...,"[Algoritmos, Algoritmos genéticos, Computação ...",[],[Inversão numérica de distribuições bidimensio...,{'Inversão numérica de distribuições bidimensi...,"[Fernando M. Ramos, Jonas C. Carvalho]","[https://orcid.org/0000-0001-5321-9805, https:...","{nan, Pedro Paulo Balbi de Oliveira, nan}"
1518,96/05497-3,"[https://orcid.org/0000-0001-5203-4366, https:...",Pesquisa em visão cibernética,O presente projeto visa suportar as atividades...,"[Inteligência artificial, Visão computacional,...",[],[Optimized approach to multiscale skeleton gen...,{'Optimized approach to multiscale skeleton ge...,"[Luciano da F. Costa, Maysa Santos Barbosa, Ri...","[https://orcid.org/0000-0001-5203-4366, https:...","{nan, nan, Luciano da Fontoura Costa}"
1519,96/04505-2,"[http://orcid.org/0000-0001-7841-157X, http://...",Aspectos estruturais e algorítmicos de objetos...,O objeto central da pesquisa proposta neste pr...,"[Algoritmos, Combinatória, Poliedros]",[https://www.doi.org/],"[Hereditary properties of triple systems, A de...",{'Hereditary p

In [78]:
df_results_from_query_simple["count_author_orcid_in_open_ai"] = df_results_from_query_simple.apply(lambda x: len(set(x["orcids"]) & set(x["all_orcids_oa"])) ,axis=1)

In [80]:
df_results_from_query_simple["has_author_orcid_in_open_ai"] = df_results_from_query_simple["count_author_orcid_in_open_ai"].apply(lambda x: x > 0)

In [99]:
print(df_results_from_query_simple[df_results_from_query_simple["count_author_orcid_in_open_ai"] == 0].iloc[[10]][["all_names_oa", "all_researchers"]]["all_names_oa"].values)

[list(['Fabio Montoni', 'Willian Wang', 'Hugo A. Armelin', 'Marcelo S. Reis'])]


In [100]:
print(df_results_from_query_simple[df_results_from_query_simple["count_author_orcid_in_open_ai"] == 0].iloc[[10]][["all_names_oa", "all_researchers"]]["all_researchers"].values)

[{nan, 'Marcelo da Silva Reis', 'Ronaldo Nogueira de Sousa', nan}]


In [88]:
df[df["N. Processo"] == "19/24580-2"]

,N. Processo,Título (Português),Título (Inglês),Beneficiário,Instituição,Cidade Instituição,Instituição Parceira,Empresa,Município,Pesquisador Responsável,...,Data de Início,Data de Término,Acordo(s)/Convênio(s) de Cooperação com a FAPESP,Instituições no Exterior,País (Inst. Ext. / Convênio),Pesquisador responsável no exterior,Resumo (Português),Resumo (Inglês),Processos Vinculados,all_researchers
751,19/24580-2,Seleção bayesiana de modelos dinâmicos de vias...,Bayesian dynamic model selection of non-isolat...,Ronaldo Nogueira de Sousa,Secretaria da Saúde (São Paulo - Estado). Inst...,São Paulo,NaN,NaN,NaN,Marcelo da Silva Reis,...,2021-10-01,2024-05-31,NaN,NaN,NaN,NaN,Uma via de sinalização celular é composta por ...,A cell signaling pathway is composed of a set ...,NaN,"{nan, Marcelo da Silva Reis, Ronaldo Nogueira ..."


In [14]:
Authors()["https://orcid.org/0000-0002-5591-3750"]

{'id': 'https://openalex.org/A5023928945',
 'orcid': 'https://orcid.org/0000-0002-5591-3750',
 'display_name': 'Jó Ueyama',
 'display_name_alternatives': ['Jó Ueyama',
  'Jo Ueyama',
  'UeyamaJo',
  'J. Ueyama'],
 'works_count': 160,
 'cited_by_count': 2365,
 'summary_stats': {'2yr_mean_citedness': 3.6666666666666665,
  'h_index': 24,
  'i10_index': 48},
 'ids': {'openalex': 'https://openalex.org/A5023928945',
  'orcid': 'https://orcid.org/0000-0002-5591-3750'},
 'affiliations': [{'institution': {'id': 'https://openalex.org/I17974374',
    'ror': 'https://ror.org/036rp1748',
    'display_name': 'Universidade de São Paulo',
    'country_code': 'BR',
    'type': 'education',
    'lineage': ['https://openalex.org/I17974374']},
   'years': [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015]},
  {'institution': {'id': 'https://openalex.org/I4210131883',
    'ror': 'https://ror.org/03kcw4w74',
    'display_name': 'Brazilian Society of Computational and Applied Mathematics',
    'cou